<a href="https://colab.research.google.com/github/zhongwenGit/test-pytorch/blob/main/pytorchNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)        # 6 filters
        self.conv2 = nn.Conv2d(6, 16, 3).      #16 filters
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1, 1, 32, 32)
print(input)
out = net(input)
print(out)

tensor([[[[-2.5171e-01,  1.9845e-01, -4.9139e-01,  ...,  1.1854e+00,
            9.3242e-02,  1.3419e-01],
          [ 4.4246e-01, -1.1092e-01,  1.6002e+00,  ...,  2.6270e-01,
           -6.8575e-01,  1.1599e+00],
          [-1.9967e+00, -3.3097e-01,  1.7932e+00,  ..., -3.9370e-01,
            1.0707e+00,  4.8554e-04],
          ...,
          [-5.0489e-01, -3.1114e-01,  6.5731e-02,  ..., -1.6091e-01,
            1.1153e+00,  8.0129e-01],
          [-1.0638e+00, -8.8413e-01,  1.1625e+00,  ..., -4.6132e-01,
            4.2671e-01,  2.0502e+00],
          [-2.7555e-01, -5.1915e-01, -8.4504e-01,  ...,  1.7746e+00,
            7.1945e-01,  1.5814e+00]]]])
tensor([[ 0.1271,  0.0831,  0.0949,  0.0738, -0.0146, -0.0173,  0.0711, -0.1381,
         -0.0419,  0.0671]], grad_fn=<AddmmBackward>)


In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [7]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0964, grad_fn=<MseLossBackward>)


In [9]:
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
 #     -> view -> linear -> relu -> linear -> relu -> linear
 #     -> MSELoss
  #    -> loss#

In [10]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [11]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0155,  0.0128,  0.0190,  0.0043, -0.0156, -0.0098])


In [12]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [13]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update